In [54]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network


In [55]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [56]:
import nest_asyncio

nest_asyncio.apply()

In [57]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [58]:
#add api key to llm

In [59]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [60]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [61]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [62]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or favorites. However, I can tell you about some popular data tools that are widely used in the industry for various purposes:

1. **Data Governance Platforms**: Tools like Collibra and Informatica offer comprehensive solutions for managing data governance, including data cataloging, data lineage, and policy management.

2. **Data Quality Tools**: Talend and Trifacta are popular for ensuring data quality by providing features for data cleansing, profiling, and transformation.

3. **Data Catalogs**: Alation and Data.world are known for their robust data cataloging capabilities, helping organizations discover and manage their data assets effectively.

4. **Data Int

# Load storage contexts

## Load vector storage context

In [63]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [64]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [65]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [66]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000001DB33F79400>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000001DB33F8D910>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x000001DB33F8DD00>, property_graph_store=None)


# Load index

## Load vector index

In [67]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [68]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [69]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan.",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [70]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, une approche structurée peut être suivie. Voici un plan détaillé :

1. **Compréhension du problème :**
   - Définir clairement l'objectif de la prédiction.
   - Identifier les variables cibles et explicatives.

2. **Collecte et préparation des données :**
   - Rassembler les données historiques des clients, y compris les informations démographiques, financières, et comportementales.
   - Nettoyer les données pour gérer les valeurs manquantes et les anomalies.
   - Effectuer une analyse exploratoire des données pour comprendre les distributions et les relations entre les variables.

3. **Sélection de la méthode de modélisation :**
   - Choisir une méthode de classification, car le problème est de nature binaire (défaut ou non).
   - Considérer des algorithmes comme la régression logistique, les arbres de décision, les forêts aléatoires, ou les machines à vecteurs de support.

4. **Construction du modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle choisi sur l'ensemble d'entraînement.
   - Optimiser les hyperparamètres pour améliorer la performance du modèle.

5. **Évaluation du modèle :**
   - Utiliser des métriques de performance telles que l'accuracy, le F1-score, la précision, et le rappel pour évaluer le modèle.
   - Effectuer une validation croisée pour assurer la robustesse du modèle.

6. **Interprétation et déploiement :**
   - Analyser les résultats pour comprendre les facteurs influençant le défaut.
   - Déployer le modèle dans un environnement de production pour des prédictions en temps réel.

7. **Surveillance et mise à jour :**
   - Surveiller la performance du modèle au fil du temps.
   - Mettre à jour le modèle régulièrement avec de nouvelles données pour maintenir sa précision.

Ce plan fournit une approche systématique pour prédire le défaut de paiement d'un client sur un prêt bancaire.</b>

## Load graph index

In [71]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [72]:
nx_graph = graph_index.get_networkx_graph()

In [73]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 6


In [74]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 9 nodes and 6 edges
Nodes: ['Model', 'Parent', 'Type_model', 'Classification', 'Clustering', 'Regression', 'Prediction', 'Type_algo', 'Optimization algorithm']
Edges: [('Model', 'Parent'), ('Type_model', 'Classification'), ('Type_model', 'Clustering'), ('Type_model', 'Regression'), ('Type_model', 'Prediction'), ('Type_algo', 'Optimization algorithm')]


### Set retriever

In [75]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [76]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan.",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 7ec05980-7510-4c

In [77]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, une méthode de classification serait appropriée. Voici un plan pour aborder ce problème :

1. **Collecte des données**: Rassemblez des données historiques sur les clients, y compris des informations démographiques, des antécédents de crédit, des revenus, etc.

2. **Prétraitement des données**: Nettoyez les données en traitant les valeurs manquantes, en normalisant les variables et en encodant les variables catégorielles.

3. **Sélection des caractéristiques**: Identifiez les caractéristiques les plus pertinentes qui influencent le défaut de paiement à l'aide de techniques de sélection de caractéristiques.

4. **Choix du modèle**: Utilisez un modèle de classification, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires, pour prédire le défaut de paiement.

5. **Entraînement du modèle**: Divisez les données en ensembles d'entraînement et de test, puis entraînez le modèle sur l'ensemble d'entraînement.

6. **Évaluation du modèle**: Évaluez la performance du modèle à l'aide de l'ensemble de test en utilisant des métriques telles que la précision, le rappel, et la courbe ROC.

7. **Optimisation**: Affinez le modèle en ajustant les hyperparamètres pour améliorer sa performance.

8. **Déploiement**: Intégrez le modèle dans le système de décision pour prédire le risque de défaut des nouveaux clients.

9. **Surveillance et mise à jour**: Surveillez la performance du modèle et mettez-le à jour régulièrement avec de nouvelles données pour maintenir sa précision.</b>

## Load onto index

In [78]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [79]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [80]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [106]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan.",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [107]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est approprié d'utiliser une méthode de classification. Voici un plan pour aborder ce problème :

1. **Compréhension du problème :**
   - Définir clairement l'objectif : prédire le défaut de paiement.
   - Identifier les variables cibles et explicatives.

2. **Collecte et préparation des données :**
   - Rassembler les données historiques des clients, y compris les caractéristiques démographiques, financières, et comportementales.
   - Nettoyer les données pour gérer les valeurs manquantes et les anomalies.

3. **Exploration des données :**
   - Analyser les données pour comprendre les distributions et les relations entre les variables.
   - Utiliser des visualisations pour identifier les tendances et les corrélations.

4. **Sélection des caractéristiques :**
   - Choisir les variables les plus pertinentes pour la prédiction.
   - Éventuellement, réduire la dimensionnalité avec des techniques comme l'analyse en composantes principales (PCA).

5. **Choix du modèle de classification :**
   - Considérer des modèles tels que la régression logistique, les arbres de décision, les forêts aléatoires, ou les machines à vecteurs de support (SVM).
   - Comparer les modèles en termes de précision, de rappel, et de courbe ROC.

6. **Entraînement et validation du modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle sur l'ensemble d'entraînement.
   - Valider le modèle avec l'ensemble de test pour évaluer sa performance.

7. **Optimisation du modèle :**
   - Ajuster les hyperparamètres pour améliorer la performance.
   - Utiliser des techniques de validation croisée pour éviter le surapprentissage.

8. **Interprétation et déploiement :**
   - Interpréter les résultats pour comprendre les facteurs influençant le défaut de paiement.
   - Déployer le modèle dans un environnement de production pour des prédictions en temps réel.

9. **Suivi et mise à jour :**
   - Surveiller la performance du modèle au fil du temps.
   - Mettre à jour le modèle avec de nouvelles données pour maintenir sa précision. 

Ce plan vous guidera dans la mise en œuvre d'une solution de classification pour prédire le défaut de paiement des clients.</b>

# Visualize knowledge graph

In [83]:
nx_graph = graph_index.get_networkx_graph()

In [84]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 6


In [85]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 9 nodes and 6 edges
Nodes: ['Model', 'Parent', 'Type_model', 'Classification', 'Clustering', 'Regression', 'Prediction', 'Type_algo', 'Optimization algorithm']
Edges: [('Model', 'Parent'), ('Type_model', 'Classification'), ('Type_model', 'Clustering'), ('Type_model', 'Regression'), ('Type_model', 'Prediction'), ('Type_algo', 'Optimization algorithm')]


# (Simple) Query the vectors

In [86]:
query = "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan."
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [87]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, une approche structurée peut être suivie. Voici un plan détaillé :

1. **Compréhension du problème :**
   - Définir clairement l'objectif de la prédiction.
   - Identifier les variables cibles et explicatives.

2. **Collecte et préparation des données :**
   - Rassembler les données historiques des clients, y compris les informations démographiques, financières, et comportementales.
   - Nettoyer les données pour gérer les valeurs manquantes et les anomalies.
   - Effectuer une analyse exploratoire des données pour comprendre les distributions et les relations.

3. **Choix de la méthode de modélisation :**
   - Utiliser une méthode de classification, car le problème est de nature binaire (défaut ou non).
   - Considérer des algorithmes comme la régression logistique, les arbres de décision, ou les forêts aléatoires.

4. **Construction du modèle :**
   - Diviser les données en ensembles d'entraînement et de test.
   - Entraîner le modèle choisi sur l'ensemble d'entraînement.
   - Ajuster les hyperparamètres pour optimiser les performances.

5. **Évaluation du modèle :**
   - Utiliser des métriques telles que l'accuracy, le rappel, la précision, et l'AUC-ROC pour évaluer la performance du modèle.
   - Comparer les performances de différents modèles pour choisir le meilleur.

6. **Interprétation et déploiement :**
   - Interpréter les résultats pour comprendre les facteurs influençant le défaut.
   - Déployer le modèle dans un environnement de production pour des prédictions en temps réel.

7. **Suivi et mise à jour :**
   - Surveiller la performance du modèle au fil du temps.
   - Mettre à jour le modèle avec de nouvelles données pour maintenir sa précision.

Ce plan permet de structurer le processus de prédiction de manière efficace et rigoureuse.</b>

# (Simple) Query the knowledge graph 

In [88]:
query = "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
usage: "[[Prediction]]"
---
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 7ec05980-7510-4c

In [89]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, une méthode de classification est appropriée. Voici un plan pour aborder ce problème :

1. **Collecte des données**: Rassemblez des données historiques sur les clients, y compris des informations démographiques, des antécédents de crédit, des revenus, etc.

2. **Prétraitement des données**: Nettoyez les données en traitant les valeurs manquantes, en normalisant les variables et en encodant les variables catégorielles.

3. **Sélection des caractéristiques**: Identifiez les caractéristiques les plus pertinentes qui influencent la probabilité de défaut.

4. **Choix du modèle**: Utilisez un modèle de classification, tel que la régression logistique, les arbres de décision, ou les forêts aléatoires, pour prédire le défaut.

5. **Entraînement du modèle**: Divisez les données en ensembles d'entraînement et de test, puis entraînez le modèle sur l'ensemble d'entraînement.

6. **Évaluation du modèle**: Évaluez la performance du modèle à l'aide de l'ensemble de test en utilisant des métriques telles que la précision, le rappel, et la courbe ROC.

7. **Optimisation**: Ajustez les hyperparamètres du modèle pour améliorer sa performance.

8. **Déploiement**: Intégrez le modèle dans le système de décision pour prédire le risque de défaut des nouveaux clients.

9. **Surveillance et mise à jour**: Surveillez la performance du modèle et mettez-le à jour régulièrement avec de nouvelles données pour maintenir sa précision.</b>

# (Simple) Query the onto graph 

In [109]:
query = "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan."
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [110]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour prédire si un client va faire défaut sur son prêt bancaire, il est approprié d'utiliser une méthode de classification. Voici un plan pour aborder ce problème :

1. **Collecte des données**: Rassembler des données historiques sur les clients, y compris des informations démographiques, financières, et des antécédents de crédit.

2. **Prétraitement des données**:
   - Nettoyage des données pour gérer les valeurs manquantes et les anomalies.
   - Normalisation ou standardisation des variables si nécessaire.
   - Encodage des variables catégorielles.

3. **Séparation des données**:
   - Diviser les données en ensembles d'entraînement et de test pour évaluer la performance du modèle.

4. **Choix du modèle de classification**:
   - Sélectionner un ou plusieurs algorithmes de classification, tels que la régression logistique, les arbres de décision, ou les forêts aléatoires.

5. **Entraînement du modèle**:
   - Utiliser l'ensemble d'entraînement pour ajuster le modèle choisi.

6. **Évaluation du modèle**:
   - Tester le modèle sur l'ensemble de test.
   - Utiliser des métriques telles que la précision, le rappel, et la courbe ROC pour évaluer la performance.

7. **Optimisation**:
   - Ajuster les hyperparamètres pour améliorer la performance du modèle.
   - Envisager l'utilisation de techniques d'ensemble pour combiner plusieurs modèles.

8. **Déploiement**:
   - Mettre en place le modèle dans un environnement de production pour prédire les défauts de paiement des nouveaux clients.

9. **Surveillance et mise à jour**:
   - Surveiller la performance du modèle au fil du temps et le mettre à jour avec de nouvelles données pour maintenir sa précision.</b>

## (Node retriever)

In [92]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classification]]"
---


# Have a real chat with your data

## Set up the engines

### Vector engines

In [93]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        " "
        " "
        "."
    ),
    verbose=False,
)

### Graph engines

In [94]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [95]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [96]:
graph_chat_engine.reset()

In [97]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        " "
        " "
        " "
    ),
    verbose=False,
)

In [98]:
response_stream = graph_chat_engine.stream_chat("""""")

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: 
Condensed question: 
Condensed question: 
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
u

In [99]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Hello! How can I assist you today?

In [100]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [101]:
section_modele = """ 


"""

In [102]:
response_stream = graph_chat_engine.stream_chat("""Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Pouvez-vous imiter le style d'écriture de la section modèle ?
Condensed question: Pouvez-vous imiter le style d'écriture de la section modèle ?
Condensed question: Pouvez-vous imiter le style d'écriture de la section modèle ?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
> Querying with idx: 4a3c46d4-923d-473c-9f33-b7c104a81217: ---
type_model:
  - "[[Clustering]]"
  - "[[Regression]]"
  - "[[Classificati...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
> Querying with idx: 34355420-d17b-4534-b492-ade4a9ad7720: ---
type_model:
  - "[[Classification]]"
---
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 20e84990-8d4c-41ad-a31d-62a63edd8a65: ---
type_model:
  - "[[Regression]]"
u

In [103]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Je suis désolé, mais il semble qu'il y ait eu une petite confusion dans votre demande. Pourriez-vous préciser quel style d'écriture ou quel modèle vous aimeriez que j'imite ? Cela m'aiderait à mieux répondre à votre demande. Merci !

In [104]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [105]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\output\output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 